<a href="https://colab.research.google.com/github/hyuckjinkim/Dacon/blob/main/%ED%95%9C%EA%B5%AD%EC%96%B4_%EB%AC%B8%EC%9E%A5_%EA%B4%80%EB%A1%80_%EB%B6%84%EB%A5%98_Ver_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

--- 

# **코드설명**

---
\

- 파 일 명 : 한국어 문장 관례 분류 Ver 1 <br>
- 시작날짜 : 2022.02.04 <br>
- 수정날짜 : 2022.02.04 <br>
- 작 성 자 : 김혁진 <br>
- 작성주제 : 한국어 문장 관례 분류 <br>

- 주제상세 : \
\
한 쌍의 문장은 Premise와 Hypothesis로 구성되어 있습니다. \
premise 문장을 참고해 hypothesis 문장이 참인지(Entailment), 거짓인지(Contradiction), \
혹은 참/거짓 여부를 알 수 없는 문장인지(Neutral)를 판별해야 합니다. 

<br>

--- 

- **참조**

  (1) 대회 홈페이지 : [Dacon](https://dacon.io/competitions/official/235875/overview/description) <br>
  (2) 하이퍼 파리미터 설명 : [Naver Blog](https://blog.naver.com/wideeyed/221333529176) <br>
  (3) Class문 설명 : [Github](https://zzsza.github.io/development/2020/07/05/python-class/) <br>
  (4) GPU 설정 : [Medium](https://medium.com/@am.sharma/lgbm-on-colab-with-gpu-c1c09e83f2af) <br>
  (5) RAM 모두사용으로 세션다운 : [Tistory](https://somjang.tistory.com/entry/Google-Colab-%EC%9E%90%EC%A3%BC%EB%81%8A%EA%B8%B0%EB%8A%94-%EB%9F%B0%ED%83%80%EC%9E%84-%EB%B0%A9%EC%A7%80%ED%95%98%EA%B8%B0)

---

- **고려사항** <br>
  (1) AutoEncoder로 파생변수 생성해보기 <br>
  (2) 하이퍼파라미터 탐색 : grid-search, bayesian-optimization, [optuna](https://dacon.io/competitions/official/235713/codeshare/2704?page=1&dtype=recent) <br>
  (3) RandomForest, XGBoost, Lightgbm, CatBoost 설명 [블로그](https://jhkim0759.tistory.com/12)

---
<br></br>

# 1.&nbsp;기본설정

\
## 1.1.&nbsp;colab 설정

In [ ]:
# # 출처: https://schulwin.tistory.com/entry/Google-Colab-설정하기-런타임-연결-끊김-방지 [Diary of Schul]
# # Google Colab을 실행시키는 크롬 브라우저 창에서 F12 (혹은 Ctrl-Shift-i)를 눌러서 개발자 도구 창을 열고 
# # Console 창에서 아래와 같이 입력을 해주면 된다.

# # (1) 런타임 연결 끊김 방지
# function ClickConnect() { 
#     // 백엔드를 할당하지 못했습니다. 
#     // GPU이(가) 있는 백엔드를 사용할 수 없습니다. 가속기가 없는 런타임을 사용하시겠습니까? 
#     // 취소 버튼을 찾아서 클릭 
#     var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); 
#     buttons.forEach(function(btn) { 
#         btn.click(); 
#     }); 
#     console.log("1분마다 자동 재연결"); 
#     document.querySelector("#top-toolbar > colab-connect-button").click(); 
# } 
# setInterval(ClickConnect,1000*60);

# # (2) Google Colab에서 30분마다 현재 출력창 지우기
# function CleanCurrentOutput(){ 
#     var btn = document.querySelector(".output-icon.clear_outputs_enabled.output-icon-selected[title$='현재 실행 중...'] iron-icon[command=clear-focused-or-selected-outputs]"); 
#     if(btn) { 
#         console.log("30분마다 출력 지우기"); btn.click(); 
#     } 
# } 
# setInterval(CleanCurrentOutput,1000*60*30);

\
## 1.2.&nbsp;Query Start Time

In [ ]:
import time
query_start_time = time.time()

\
## 1.3.&nbsp;Markdown : Tabular Left Align

In [ ]:
%%html
<style>
    table {float:left}
</style>

\
## 1.4.&nbsp;Jupyter Notebook Style : Theme, Display

In [ ]:
# # theme 설치
# !pip install jupyterthemes

# # jupyter notebook 최신버전
# !pip install --upgrade notebook

# # jupyter notebook 최신버전
# !pip install --upgrade jupyterthemes

# 2.2.1. 테마바꾸기(customizing)
# !jt -t onedork -fs 115 -nfs 125 -tfs 115 -dfs 115 -ofs 115 -cursc r -cellw 80% -lineh 115 -altmd  -kl -T -N

# 2.2.2. 쥬피터 노트북 화면 넓게 사용
# 출처: https://taehooh.tistory.com/entry/Jupyter-Notebook-주피터노트북-화면-넓게-쓰는방법
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:80% !important; }</style>"))

# # 2.2.3. 좌측 TOC 만들기
# # 출처 : https://gmnam.tistory.com/246
# !pip install jupyter_nbextensions_configurator
# !pip install jupyter_contrib_nbextensions

# !jupyter nbextensions_configurator enable --user
# !jupyter contrib nbextension install --user

In [ ]:
# # 2.3.1 Google Drive Mount
# # (Google Drive 사용 시 설정)
from google.colab import drive
import os, sys

# # 새로운 창에서 key 를 받아서 입력해야합니다.
# drive.mount('/content/drive') # , force_remount = True

# colab 패키지 영구적으로 저장
# 심볼릭링크 : https://com-flex.tistory.com/12
pg_path = '/content/packages'  
os.symlink('/content/drive/MyDrive/Colab Notebooks', pg_path) 
sys.path.insert(0, pg_path)  

# # 2.3.2. 메모리 에러
# https://growingsaja.tistory.com/477

In [ ]:
# # 2.3.3. GPU 사용 (6분)
# !git clone --recursive https://github.com/Microsoft/LightGBM
# !mkdir build
# %cd /content/LightGBM
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
# %cd /content/LightGBM/python-package

In [ ]:
# !pip install --target=$pg_path pandasql
# !pip install --target=$pg_path seaborn
# !pip install --target=$pg_path plotnine
# !pip install --target=$pg_path pandasql

# # lightgbm 에러떴는데, 콘다에서 실행하면 해결됨
# # conda install -c conda-forge lightgbm 

# # bayesian optimization 설치
# !pip install --target=$pg_path bayesian-optimization

# # xgboost 설치
# !pip install --target=$pg_path xgboost

# # catboost 설치
# !pip install --target=$pg_path catboost

# !pip install --target=$pg_path tall_path dill

# # pycaret 에러떴는데, --user 붙이니깐 해결됨
# !pip install --target=$pg_path --user pycaret

In [ ]:
# !pip install --target=$pg_path lightgbm --install-option=--gpu

# !pip install transformers # --target=$pg_path

# !pip install --user optuna # --target=$pg_path

\
## 1.5.&nbsp;Import Modules

In [ ]:
import optuna

In [ ]:
#------------------------------------------------------------------------------#
# jupyter notebook 전용
#------------------------------------------------------------------------------#
# from tqdm.notebook import tqdm
from tqdm import tqdm

#------------------------------------------------------------------------------#
# basic modules
#------------------------------------------------------------------------------#
import pandas as pd
import numpy as np
import math
import warnings
import random
import os
import time
import itertools

# value_counts() 범용적인 버전
from collections import Counter as cnt

# # sqldf
# from pandasql import sqldf
# sql = lambda q: sqldf(q, globals())

#------------------------------------------------------------------------------#
# save env.
#------------------------------------------------------------------------------#
# import dill

# #------------------------------------------------------------------------------#
# # plotting
# #------------------------------------------------------------------------------#
# import seaborn as sns
# sns.set(rc={'figure.figsize':(11.7, 8.27)})
# sns.set_style('whitegrid')

# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams['figure.figsize'] = [11.7, 8.27] # [15, 10] # [11.7,8.27] - A4 size
# plt.style.use('dark_background')

# from plotnine import *

# #------------------------------------------------------------------------------#
# # modeling
# #------------------------------------------------------------------------------#
# import sklearn
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder, PolynomialFeatures
# from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score,StratifiedKFold,train_test_split
# from sklearn.metrics import f1_score,make_scorer,r2_score
# from sklearn.compose import ColumnTransformer

# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import KFold
# from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeClassifier, Ridge, Lasso, ElasticNet
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.svm import SVR
# from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor, AdaBoostClassifier
# from sklearn.metrics import mean_squared_error
# from xgboost import XGBRegressor, XGBClassifier
# from lightgbm import LGBMRegressor, LGBMClassifier
# import lightgbm as lgb
# from catboost import CatBoostRegressor, CatBoostClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC

from transformers import pipeline, AutoTokenizer

# # Hyperparameter Optimization
# from bayes_opt import BayesianOptimization
# import optuna

\
## 1.6.&nbsp;Initial Values

In [ ]:
# 2.5.1. Data Path
DATA_PATH = '/content/drive/MyDrive/Python/4. 한국어 문장 관계 분류/DAT/'
OUT_PATH  = '/content/drive/MyDrive/Python/4. 한국어 문장 관계 분류/OUT/'
PARA_PATH = '/content/drive/MyDrive/Python/4. 한국어 문장 관계 분류/best_para/'

# 2.5.2. set seed
SEED = 777

\
## 1.7.&nbsp;Set Off the Warning

In [ ]:
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')

\
## 1.8.&nbsp;User Defined Function

In [ ]:
#-------------------------------------------------------------------------------------------------------#
# 2.6.1. Seed Fix
#-------------------------------------------------------------------------------------------------------#
def seed_everything(seed: int = 1):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)  # type: ignore
    # torch.backends.cudnn.deterministic = True  # type: ignore
    # torch.backends.cudnn.benchmark = True  # type: ignore
    
seed_everything(SEED)

#-------------------------------------------------------------------------------------------------------#
# 2.6.2. View all columns
#-------------------------------------------------------------------------------------------------------#
def View(data):

    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    
    print(data)

    pd.set_option('display.max_rows', 0)
    pd.set_option('display.max_columns', 0)
    pd.set_option('display.width', 0)

#-------------------------------------------------------------------------------------------------------#
# 2.6.3. minmax function
#-------------------------------------------------------------------------------------------------------#
def minmax(x, digit=None):
    if round is None:
        return min(x),max(x)
    else:
        return round(min(x),digit),round(max(x),digit)

#-------------------------------------------------------------------------------------------------------#
# 2.6.4. 컬럼dict에서 target 제거
#-------------------------------------------------------------------------------------------------------#
# - dict : 기준 dict
# - key  : 삭제할 key
#-------------------------------------------------------------------------------------------------------#
def rmkey(dict, key):
    tmp = dict.copy()
    del tmp[key]
    return tmp

#-------------------------------------------------------------------------------------------------------#
# 2.6.5. 각 컬럼의 missing 개수를 파악하는 함수
#-------------------------------------------------------------------------------------------------------#
# - data     : 기준 data
# - col_type : {column명 : type}로 이루어진 dictionary
#-------------------------------------------------------------------------------------------------------#
def missing_column_check(df, col_type):
    
    data = df.copy()
    
    n_na = []
    n_na_type = []
    for col_nm in data.columns:
        data[col_nm] = data[col_nm].astype(col_type[col_nm])

        # str인 경우에는 blank(공백)도 있는지 확인
        if col_type[col_nm]==str:

            isnull_cnt = data[col_nm].str.strip().isnull().sum()
            blank_cnt  = sum(data[col_nm].str.strip()=='')
            nan_cnt    = sum(data[col_nm].str.strip()=='nan')
            null_cnt   = sum(data[col_nm].str.strip()=='null')

            n_na_x = isnull_cnt+blank_cnt+nan_cnt+null_cnt
            n_na.append(n_na_x)
            
            if n_na_x>0:
                n_na_type_x=[]
                if isnull_cnt>0: n_na_type_x.append('isnull')
                if blank_cnt >0: n_na_type_x.append('blank')
                if nan_cnt   >0: n_na_type_x.append('nan')
                if null_cnt  >0: n_na_type_x.append('null')
                n_na_type_x = '+'.join(n_na_type_x)
            else:
                n_na_type_x = ''
            n_na_type.append(n_na_type_x)
            

        # numeric인 경우에는 null의 개수만 확인
        else:
            n_na_x = data[col_nm].isnull().sum()
            n_na.append(n_na_x)
            
            if n_na_x>0:
                n_na_type.append('isnull')
            else:
                n_na_type.append('')
            
    res_df = pd.DataFrame({
        'col'  : data.columns,
        'n_na' : n_na,
        'n_n_ratio' : [str(round(n/len(data)*100,1))+'%' for n in n_na],
        'na_type' : n_na_type,
        'col_type' : [COL_TYPE[col].__name__ for col in data.columns]
        })

    res_df = res_df[res_df['n_na']>0]
    if len(res_df)==0:
        return('Dataset does not have a null value')
    else:
        return(res_df)

#-------------------------------------------------------------------------------------------------------#
# 2.6.6. 교호작용항 추가
#-------------------------------------------------------------------------------------------------------#
# - data     : 기준 data
# - num_vari : 숫자형 변수 list
#-------------------------------------------------------------------------------------------------------#
def interaction_term(df,num_vari):
    
    data = df.copy()

    num_var = list(set(num_vari) - set(['id']))

    for i in range(0,len(num_var)):
        for j in range(i,len(num_var)):
            data[f'{num_var[i]}*{num_var[j]}'] = data[f'{num_var[i]}']*data[f'{num_var[j]}']

    return(data)

#-------------------------------------------------------------------------------------------------------#
# 2.6.7. color when print
#-------------------------------------------------------------------------------------------------------#
class color:
    PURPLE    = '\033[95m'
    CYAN      = '\033[96m'
    DARKCYAN  = '\033[36m'
    BLUE      = '\033[94m'
    GREEN     = '\033[92m'
    YELLOW    = '\033[93m'
    RED       = '\033[91m'
    BOLD      = '\033[1m'
    UNDERLINE = '\033[4m'
    END       = '\033[0m'

#-------------------------------------------------------------------------------------------------------#
# 2.6.8. density plot : histogram + density plot
#-------------------------------------------------------------------------------------------------------#
# - data : 기준 data
# - vars : hist + kde를 그릴 숫자형 변수
# - hue  : group화 변수
# - binwidth_adj_ratio : binwidth 조정 비율
#-------------------------------------------------------------------------------------------------------#
def density_plot(df, vars, 
                 binwidths = None, hue = None,
                 binwidth_adj_ratio = None):

    from matplotlib.ticker import PercentFormatter
    
    data = df.copy()

    # 1) vars가 1개뿐일 때 에러발생
    #    -> 1개     : type = str
    #    -> 2개이상 : type = ndarray, ...
    if type(vars)==str:
        vars = [vars]
    
    # 2) plotting (nrow,ncol) 설정
    nrow = math.ceil(len(vars)**(1/2))
    ncol = nrow

    # 3) binwidths가 없을 때, binwidth 설정
    # 출처 : http://www.aistudy.co.kr/paper/pdf/histogram_jeon.pdf
    if binwidths is None:
        binwidths = []
        for col in data[vars].columns:
            n_bin = math.ceil(1 + 3.32*math.log10(len(data)))
            binwidth = ( data[col].max() - data[col].min() ) / n_bin
            binwidths.append(binwidth)
            del binwidth
    
    # 4) 설정한 binwidth를 조정하는 비율
    if binwidth_adj_ratio is not None:
        binwidths = [binwidth * binwidth_adj_ratio for binwidth in binwidths]
    
    fig = plt.figure()
    
    # 5) vars 별로 plot 생성
    for iter,var in enumerate(vars):
        
        binwidth = binwidths[iter]
        
        # (1) histogram
        ax1 = fig.add_subplot(nrow, ncol, iter+1)
        g1 = sns.histplot(data = data, x = var, hue = hue,
                          kde = True, stat = 'probability', 
                          color = 'lightskyblue',
                          binwidth = binwidth, ax = ax1)
        ax2 = ax1.twinx()
        
        # (2) density plot
        g2 = sns.kdeplot(data = data, x = var, hue = hue,
                         color = 'red', lw = 2, ax = ax2)
        ax2.set_ylim(0, ax1.get_ylim()[1] / binwidth)                  # similir limits on the y-axis to align the plots
        #ax2.yaxis.set_major_formatter(PercentFormatter(1 / binwidth))  # show axis such that 1/binwidth corresponds to 100%
        ax2.grid(False)
        
        # (3) density plot y축 없애기
        g2.set(yticklabels=[]) 
        g2.set(ylabel=None)
        g2.tick_params(right=False)
        
        a,b = divmod(iter,ncol)
        if b!=0:
            g1.set(ylabel=None)
        
    # 안겹치도록 설정
    fig.tight_layout()
    plt.show()

# example : density_plot(train, vars=num_vari)

#-------------------------------------------------------------------------------------------------------#
# 2.6.9. density plot : histogram + density plot
#
# (1) grp_var vs hue_var 막대그래프
# (2) grp_var(x축), hue_var에 따른 각 num_var들의 barplot, violineplot, box+swarmplot + kdeplot
#-------------------------------------------------------------------------------------------------------#
#- grp_var : x축 구분할 그룹변수 (text)
#- num_vari : 숫자형 변수 (list)
#- data : 기준 data
#- title_text : plot title (text)
#- hue_var : hue 그루핑변수
#-------------------------------------------------------------------------------------------------------#
def plot_num(grp_var, num_vari, df, title_text=None, hue_var=None):
    
    data = df.copy()
    
    # (1)번 그래프 setting
    fig0 = plt.figure(figsize=(3,3))
    ax0  = fig0.add_subplot(1,1,1)
    
    if title_text is None:
        title_text = grp_var
        
    plt.title(title_text, loc='left', pad=20, fontdict={'fontsize' : 30,
                                                        'fontweight' : 'bold',
                                                        'color' : 'c'})
    
    # grp_var와 hue_var가 겹치는 경우, hue를 나누지 않음
    if (grp_var!=hue_var) and (hue_var is not None):

        ct = pd.crosstab(data[grp_var],data[hue_var])
        ax = ct.plot(kind='bar', stacked=False, rot=0, ax=ax0)
        ax.legend(title=hue_var, bbox_to_anchor=(1, 1.02), loc='upper left')
        
    else:
        ct = data[grp_var].value_counts()
        ax = ct.plot(kind='bar', stacked=False, rot=0, ax=ax0)
        
    # show
    plt.xlabel('')
    plt.show()
    
    # 숫자변수중에 [grp,id]변수가 있으면 제외
    num_vari_x = list(set(num_vari) - set([grp_var,'id']))
    
    # plt 생성
    fig = plt.figure(figsize=(15,15)) # figsize=(15,7)
    plt.axis('off') # 안끄면 x축에 0~1까지 축생김
    
    for iter,var in enumerate(num_vari_x):

        # hue랑 grp_var랑 같으면 hue를 넣지않음
        hue_x = [None if grp_var==hue_var else hue_var][0]

        # (n,4) plot
        ax1 = fig.add_subplot(len(num_vari_x),4,4*iter+1)
        ax2 = fig.add_subplot(len(num_vari_x),4,4*iter+2)
        ax3 = fig.add_subplot(len(num_vari_x),4,4*iter+3)
        ax4 = fig.add_subplot(len(num_vari_x),4,4*iter+4)

        #---------------------------------------------------------------------------------------------
        # (2-1) 3번째 : box + swarm plot (ylim가져오기위해서 제일 먼저 실행)
        #---------------------------------------------------------------------------------------------
        g11=sns.swarmplot(x=grp_var, y=var, data=data, ax = ax3, color='crimson', marker='*', s = 7)
        g12=sns.boxplot  (x=grp_var, y=var, data=data, ax = ax3)
        g12.set(ylabel=None)
        g12.set(yticklabels=[])

        #---------------------------------------------------------------------------------------------
        # (2-2) 1번째 : barplot
        #---------------------------------------------------------------------------------------------
        ax1.set_ylim(ax3.get_ylim())
        g21=sns.barplot(x=grp_var, y=var, data=data, ax=ax1, hue=hue_x)
        # g21.set(ylabel=None)
        # g21.set(yticklabels=[])
        # g21.axes.set_title(str(iter+1) + ':' + var, fontsize=20, weight='bold', ha='left', x=-.05)
        g21.set_ylabel(var,fontsize=20)

        #---------------------------------------------------------------------------------------------
        # (2-3) 2번째 : violinplot
        #---------------------------------------------------------------------------------------------
        ax2.set_ylim(ax3.get_ylim())
        g31=sns.violinplot(x=grp_var, y=var, data=data, ax=ax2, legend=False, hue=hue_x)
        g31.set(ylabel=None)
        g31.set(yticklabels=[])

        #---------------------------------------------------------------------------------------------
        # (2-4) 4번째 : density plot
        #---------------------------------------------------------------------------------------------
        ax4.set_ylim(ax3.get_ylim())
        g41=sns.kdeplot(y=var, hue=grp_var, data=data, ax=ax4)
        g41.set(ylabel=None)
        g41.set(yticklabels=[])
        g41.tick_params(right=False)
        g41.set(xlabel=None)
        g41.set(xticklabels=[])
        
        # 맨 아래에만 x축이 생성되도록 setting
        if (iter+1) != len(num_vari_x):
            
            g12.set(xlabel=None)
            g12.set(xticklabels=[])

            g21.set(xlabel=None)
            g21.set(xticklabels=[])

            g31.set(xlabel=None)
            g31.set(xticklabels=[])

    fig.tight_layout()
    plt.show()

# # example
# plot_num(grp_var = 'sex', num_vari = num_vari, hue_var = 'target',
#          data = train, title_text = 'sex')


#-------------------------------------------------------------------------------------------------------#
# 2.6.10. onehot encoding
#-------------------------------------------------------------------------------------------------------#
#- data
#- col_types
#-------------------------------------------------------------------------------------------------------#
def onehot_encoding(data, col_types):

    raw_data = data.copy()
    
    cols = list(set(data.columns) - set(['target']))

    for col in cols:
        if col_types[col]==str:

            data = pd.concat([
                data.drop([col],axis=1).reset_index(drop=True),
                pd.get_dummies(data[col], prefix = col).reset_index(drop=True).apply(lambda x:x.astype(float))
                ],
                axis=1)
    
    return(data)

#-------------------------------------------------------------------------------------------------------#
# 2.6.10. onehot encoding : str들 모두 int/category로 바꾸기
#-------------------------------------------------------------------------------------------------------#
#- data
#- col_types
#- convert : int / category
#-------------------------------------------------------------------------------------------------------#
def str_convert(data, col_types, convert = [int,'category']):

    cols = list(set(data.columns) - set(['target']))

    for col in cols:
        if col_types[col]==str:
            data[col] = data[col].astype(convert)
    
    return(data)

#-------------------------------------------------------------------------------------------------------#
# 2.6.11. setdiff
#-------------------------------------------------------------------------------------------------------#
def setdiff(x,y):
    return(list(set(x)-set(y)))

\
## 1.9.&nbsp;버전 확인

In [ ]:
import sys
print(sys.version)

\
# 2.&nbsp;Data Load

In [ ]:
train = pd.read_csv(DATA_PATH + 'train_data.csv')
test  = pd.read_csv(DATA_PATH + 'test_data.csv')
sub   = pd.read_csv(DATA_PATH + 'sample_submission.csv')

In [ ]:
train.info()

In [ ]:
example_num = 1001
print(f'{color.CYAN}> premise\t{color.END}: {train["premise"][example_num]}')
print(f'{color.CYAN}> hypothesis\t{color.END}: {train["hypothesis"][example_num]}')
print(f'{color.CYAN}> label\t\t{color.END}: {train["label"][example_num]}\n')
train.head(5)

In [ ]:
print(train.isnull().values.any()) # Null 값이 존재하는지 확인

# train = train.dropna(how='any') # Null 값이 존재하는 행 제거
# train = train.reset_index(drop=True)

In [ ]:
print(f'{color.CYAN}> train{color.END}')
print(f'premise    최소,최대 길이 : ({train["premise"].map(len).min()},{train["premise"].map(len).max()})')
print(f'hypothesis 최소,최대 길이 : ({train["hypothesis"].map(len).min()},{train["hypothesis"].map(len).max()})')

print('')

print(f'{color.CYAN}> test{color.END}')
print(f'premise    최소,최대 길이 : ({test["premise"].map(len).min()},{test["premise"].map(len).max()})')
print(f'hypothesis 최소,최대 길이 : ({test["hypothesis"].map(len).min()},{test["hypothesis"].map(len).max()})')

In [ ]:
print(f'\n{"-"*200}\n')

print(f'{color.CYAN}> premise min ({train["premise"].map(len).min()}){color.END}')
print(f'premise    : {train["premise"][train["premise"].map(len).argmin()]}')
print(f'hypothesis : {train["hypothesis"][train["premise"].map(len).argmin()]}')
print(f'label      : {train["label"][train["premise"].map(len).argmin()]}\n')

print(f'{color.CYAN}> premise max ({train["premise"].map(len).max()}){color.END}')
print(f'premise    : {train["premise"][train["premise"].map(len).argmax()]}')
print(f'hypothesis : {train["hypothesis"][train["premise"].map(len).argmax()]}')
print(f'label      : {train["label"][train["premise"].map(len).argmax()]}')

print(f'\n{"-"*200}\n')

print(f'{color.CYAN}> hypothesis min ({train["hypothesis"].map(len).min()}){color.END}')
print(f'premise    : {train["premise"][train["hypothesis"].map(len).argmin()]}')
print(f'hypothesis : {train["hypothesis"][train["hypothesis"].map(len).argmin()]}')
print(f'label      : {train["label"][train["hypothesis"].map(len).argmin()]}\n')

print(f'{color.CYAN}> hypothesis min ({train["hypothesis"].map(len).max()}){color.END}')
print(f'premise    : {train["premise"][train["hypothesis"].map(len).argmax()]}')
print(f'hypothesis : {train["hypothesis"][train["hypothesis"].map(len).argmax()]}')
print(f'label      : {train["label"][train["hypothesis"].map(len).argmax()]}')

print(f'\n{"-"*200}\n')

\
# 3.&nbsp;Modelling - BERT by optuna

- What is BERT : [Link1](https://riverkangg.github.io/nlp/nlp-bertWordEmbedding/), [Link2](https://ebbnflow.tistory.com/151), [Link3](https://riverkangg.github.io/nlp/nlp-bertWordEmbedding/) \

- model 선택 : [huggingface](https://huggingface.co/) \
- with optuna : [github](https://thigm85.github.io/blog/search/cord19/bert/transformers/optuna/2020/11/07/bert-training-optuna-tuning.html)

In [ ]:
TEST_SIZE = 0.25
model_name_list = [
  "klue/roberta-base",
  "Huffon/klue-roberta-base-nli",
  "roberta-large-mnli",          
]
model_name = model_name_list[1]

In [ ]:
import tensorflow as tf
print(f'{color.CYAN}is_gpu_avlilable     {color.END}: {tf.test.is_gpu_available()}')
print(f'{color.CYAN}gpu_device_name      {color.END}: {tf.test.gpu_device_name()}')
print(f'{color.CYAN}gpu_device_name_list {color.END}: {tf.config.experimental.list_physical_devices(device_type="GPU")}')

In [ ]:
X = train.drop('label',axis=1)
y = train['label']

X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=TEST_SIZE, random_state=SEED)

In [ ]:
from transformers import EvalPrediction
import numpy as np

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

from transformers import BertForSequenceClassification, Trainer, TrainingArguments

def model_init():
    model = BertForSequenceClassification.from_pretrained(model_name)
    for param in model.base_model.parameters():
        param.requires_grad = False
    return model

def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 20),
        "seed": trial.suggest_int("seed", 1, 40),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
    }
def my_objective(metrics):
    return metrics["eval_loss"]

In [ ]:
trainer = Trainer(
    train_dataset=X_tr,               # training dataset
    eval_dataset=X_va,                # evaluation dataset
    compute_metrics=compute_metrics,  # metrics to be computed
    model_init=model_init             # Instantiate model before training starts
)

In [ ]:
import optuna
best_run = trainer.hyperparameter_search(direction="minimize", hp_space=my_hp_space, compute_objective=my_objective, n_trials=100)

In [ ]:
with open(PARA_PATH + "best_run.json", "w+") as f:
  f.write(json.dumps(best_run.hyperparameters))